In [1]:
import os
import PyPDF2
import re
import pandas as pd
import numpy as np

C:\Users\Rocky\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Rocky\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
reader = PyPDF2.PdfReader("sample_problem.pdf")
num_pg = len(reader.pages)

#start_no = 2
#end_no = num_pg - 1

for pg in range(0, 2):
    page = reader.pages[pg]
    try:
        file = open('pdf_content.txt','a')
    except FileNotFoundError:
        file = open('pdf_content.txt','w')
        
    file.write(page.extract_text())
    file.close()
    
out_file = open('pdf_line_content.txt','w')
new_file = open('pdf_content.txt','rb')

s = new_file.read()
strn = re.split(' No', str(s))
out_file.write('\n'.join(strn))

new_file.close()
out_file.close()

out_files = open('pdf_line_content.txt','r')

row = []

for eachline in out_files.readlines():
    name = re.findall(r'.\dName\s:\s(.*?)\s', eachline)
    relative = re.findall(r'\sName\s:\s(.*?)\s\s\d\d', eachline)
    gender = re.findall(r'.Gender\s:\s(.*?)Age', eachline)
    age = re.findall(r'[A-Z]*\s\s(\d\d)\s.\w', eachline)
    h_no = re.findall(r'\s:\s(.*?)Gender', eachline)
    
    row.append((name, relative, gender, age, h_no))
    
out_files.close()
os.remove('pdf_content.txt')
os.remove('pdf_line_content.txt')

df = pd.DataFrame(row, columns = ['Name', 'Relative Name','Gender','Age','House No.'])

for col in df.columns:
    df[col] = df[col].apply(lambda i: ''.join(i))
    
df.replace('', np.nan, inplace=True)
df.dropna(how='all', inplace=True)

writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer, 'Content')

writer.close()